In [63]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
from os import listdir

#music imports
from music21 import converter, key
import pretty_midi
# folder
data_dir = './data/'

## METADATA about the dataset
Summary: lots of beethoven, most common instrument is solo piano but there is also string, and woodwind instruments.

In [23]:
# data
metadata_df = pd.read_csv(data_dir+"musicnet_metadata.csv")
print("number of data entries", len(metadata_df))
# print("features", metadata_df.columns.to_list())
metadata_df.head()

In [24]:
#plot all of these in one figure
fig, axs = plt.subplots(3,1, figsize=(10, 10))
metadata_df['composer'].value_counts().head(10).plot(kind='barh', ax=axs[0])
metadata_df['source'].value_counts().head(10).plot(kind='barh', ax=axs[1])
metadata_df['ensemble'].value_counts().head(10).plot(kind='barh', ax=axs[2])

## Understanding the data (.mid files)
First I organized all of the filenames under each composer in my data. Then I explored how to extract information from each of the .mid files. These files store the music information for the pieces.

In [64]:
#save the different values
composer_names = metadata_df['composer'].unique()
composer_files = {}
for composer_name in composer_names:
    composer_dir = data_dir+"musicnet_midis/musicnet_midis/"+composer_name
    composer_files[composer_name] = [composer_dir+"/"+fname for fname in listdir(composer_dir)]

print(composer_files)
print(composer_names)

In [62]:
#preview the .mid data in a given filename
test_file = composer_files['Bach'][0]


# Load MIDI file
midi_data = pretty_midi.PrettyMIDI(test_file)

# Get tempo(s)
tempo, beat_times = midi_data.get_tempo_changes()
print("Tempo (BPM):", tempo)

# Instruments
for instrument in midi_data.instruments:
    print(f"Instrument: {instrument.name}, Program: {instrument.program}, Drum: {instrument.is_drum}")
    # for note in instrument.notes[:10]:  # Print first 10 notes
    #     print(f"Note: {note.pitch}, Start: {note.start:.2f}, End: {note.end:.2f}, Velocity: {note.velocity}")

# Duration
print("Total duration (seconds):", midi_data.get_end_time())

# Key analysis with music21 (optional)


score = converter.parse(test_file)
k = score.analyze('key')
print("Estimated Key:", k)

for instrument in midi_data.instruments:
    start_times = [note.start for note in instrument.notes]
    pitches = [note.pitch for note in instrument.notes]
    plt.scatter(start_times, pitches, alpha=0.6, label=instrument.name)

plt.xlabel("Time (s)")
plt.ylabel("MIDI Pitch")
plt.title("Note Onsets")
plt.legend()
plt.show()




In [ ]:
def process_mid_file(filename):
    midi_data = pretty_midi.PrettyMIDI(filename)
    tempo, beat_times = midi_data.get_tempo_changes()
    duration =  midi_data.get_end_time()
    score = converter.parse(test_file)
    key = score.analyze('key')

    # instrument info
    instruments_data =[ ]
    for instrument in midi_data.instruments:
        start_times = [note.start for note in instrument.notes]
        pitches = [note.pitch for note in instrument.notes]

        # save all he info of the instrument
        instrument_info = {'name':instrument.name,
         'program':instrument.program,
         'drum':instrument.is_drum,
         'start_times':start_times,
         'pitches':pitches
         }
        instruments_data.append(instrument_info)
        
    # save it all in an object to summarizee
    file_data = {
        ''
    }
    